In [ ]:
import sys
sys.path.insert(0, './src/')

import nwq2qis
from nwq2qis import *

##
import numpy as np
import qiskit
import qiskit.quantum_info as qi
from qiskit_nature.second_q.mappers import JordanWignerMapper, InterleavedQubitMapper
from _gcim_utilis import parse_hamiltonian
import scipy.sparse.linalg as ssl
import scipy.linalg as sl
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
import json
from qiskit_ibm_runtime import RuntimeEncoder
from qiskit.primitives import StatevectorEstimator
##

n_orb = 6
n_a = 3
n_b = 3
ducc_lvl = 3
results_file = '../Qubit-ADAPT-VQE/printout_qubit_bztz66ducc3.txt'
output_folder = 'Example_Outputs/Benzene'  ## Please create this folder before you run the code
noise_factors = [1,3,5]
selected_iter = 1
FIRST_NTERMS = 39
mol_name =  f'Benzene{n_a+n_b}{n_orb}DUCC{ducc_lvl}'
file_path = "../Benzene/cc-pVTZ/FrozenCoreCCSD_6Elec_6Orbs/DUCC3/benzene-FC-CCSD-6Electrons-6Orbitals.out-xacc"

print("  - Number of electrons:", n_a + n_b)
print("  - Number of orbitals:", n_orb)
print("  - DUCC:", ducc_lvl)
print("  - Target molecule:", mol_name)
print("  - Input file:", file_path)

fermi_ham = parse_hamiltonian(file_path, n_orb, interleaved=False, return_type = 'qiskit')
nwqsim_dict = extract_nwqsim_output(results_file, inverse_pauli=False)

mapper = JordanWignerMapper()
qubit_op = mapper.map(fermi_ham)

true_eigvals = ssl.eigsh(qubit_op.to_matrix(sparse=True),2,which='SA')[0]

errors = np.abs( np.array(nwqsim_dict['energies']) - true_eigvals[0] )
for i, error in enumerate(errors):
    print(f"{i}:  {str(nwqsim_dict['operators'][i].paulis[0])}   {error}")

  - Number of electrons: 6
  - Number of orbitals: 6
  - DUCC: 3
  - Target molecule: Benzene66DUCC3
  - Input file: Input_Data/bztz66ducc3.hamil
0:  IIXYIIIIXXII   0.013488393702175472
1:  IXZZYIIXZZXI   0.00906966441732493
2:  IXYYYIIIIIII   0.006903987497963726
3:  IIIIIIIXYYYI   0.0046278371532935125
4:  IIXZYIIXZXII   0.003211208185632586
5:  IXZYIIIIYZYI   0.0016633754947577017
6:  IXZYIIIYZYII   0.0009027892714073005
7:  IIXZYIIIXZXI   5.970698782675754e-05
8:  XZZZZYXZZZZX   4.01313902500533e-05
9:  IXZZZYIXZZZX   1.9668743874490247e-05
10:  IIXZZYIIYZZY   8.206700982782422e-06
11:  XZZZYIYZZZYI   6.411620262269935e-06
12:  XZZYIIXZZXII   4.660086062813207e-06
13:  IXZZXIIIXYII   2.523686958966209e-06
14:  IIXXIIIYZZXI   1.889177326575009e-06
15:  IYZZZYIXZZZY   1.8120476568128652e-06
16:  IXZZZYIYZZZY   1.5580926060465572e-06
17:  IIYZZYIIYZZX   1.2352656995062716e-06


## CHOP Hamiltonian

In [2]:
## remove values that too small
np.set_printoptions(suppress=True)
shot_factor = 5000 ## Seem like 5000 is a sweet spot for accuracy

## Compute sum of coefficients
pas, cas = pauli_coeff_separation(qubit_op)
ck_abs = np.abs(cas[1:]) ## 1st term is too large
ck_sum = np.linalg.norm(ck_abs, ord=1)
chop_thresh =  ck_sum / shot_factor
print(chop_thresh)
# chop_thresh = 1e-5
qubit_op_chopped = qubit_op.chop(chop_thresh)
print(f"Chopped/unchopped # of Pauli strings  = {len(qubit_op_chopped)}/{len(qubit_op)}")

pas_ch, cas_ch = pauli_coeff_separation(qubit_op_chopped)

0.0011622583164849999
Chopped/unchopped # of Pauli strings  = 143/371


In [ ]:
pauli_dict, coeff_dict = grouped_pauli_in_dict(qubit_op_chopped, use_qwc=True)


ham_dict_ch = { pp:cc for pp,cc in zip(pas_ch, cas_ch)}

test = SparsePauliOp(list(ham_dict_ch.keys()), list(ham_dict_ch.values()) )
test.equiv(qubit_op_chopped)

True

## Optimize Parameters

In [ ]:
import nwq2qis
import importlib
importlib.reload(nwq2qis)
from nwq2qis import *

vqe_circ, symbol_params = create_vqe_circ(nwqsim_dict['operators'][:selected_iter+1], 
                                            n_orb*2, n_a+n_b, interleaved=False, half_barriar=True)

## Optimize parameters
def cost_func(parameters):
    """Compute the energy for given parameters."""
    estimator = StatevectorEstimator()
    job = estimator.run([(vqe_circ, qubit_op_chopped, parameters)])
    return float( job.result()[0].data.evs )

num_vars = len(nwqsim_dict['parameters'][:selected_iter+1])
optimizer = COBYLA(maxiter=5000, disp=True)
result = optimizer.minimize(cost_func, nwqsim_dict['parameters'][:selected_iter+1])

pre_opt_params = result.x

basis_gates = ['u', 'cx']
trans_vqe_opt = transpile(vqe_circ.assign_parameters(pre_opt_params), basis_gates=basis_gates, 
                          optimization_level=2, seed_transpiler=7)
print(trans_vqe_opt.count_ops(), cost_func(pre_opt_params), nwqsim_dict['energies'][selected_iter])

np.save(f"{output_folder}/optparam-vqe-itr{selected_iter}.npy", pre_opt_params)
pre_opt_params


   Normal return from subroutine COBYLA

   NFVALS =   36   F =-2.317783E+02    MAXCV = 0.000000E+00
   X =-6.274584E-01  -6.273679E-01
OrderedDict({'u': 22, 'cx': 20, 'barrier': 1}) -231.77831320185896 -231.77871248412274


array([-0.62745842, -0.62736795])

In [6]:
trans_vqe_opt.count_ops(), trans_vqe_opt.depth()

(OrderedDict([('u', 22), ('cx', 20), ('barrier', 1)]), 26)

## Choose First 39 groups that has the largest sum of coefficients

In [8]:
import nwq2qis
import importlib
importlib.reload(nwq2qis)
from nwq2qis import *

pauli_dict, coeff_dict = grouped_pauli_in_dict(qubit_op_chopped, use_qwc=True)
## Select first N Groups depends on the sum of abs of coefficients
coeffs_selected = dict_sort_and_select(coeff_dict,FIRST_NTERMS)
pauli_selected = {k:v for k,v in pauli_dict.items() if k in coeffs_selected.keys()}
## Construct Partial Hamiltonian
counter = 0
partial_paulis = []
partial_coeffs = []
partial_hamop = None
for k in pauli_selected.keys():
    sub_ham = qiskit.quantum_info.SparsePauliOp(pauli_selected[k], coeffs_selected[k])
    if partial_hamop is None:
        partial_hamop = sub_ham
    else:
        partial_hamop += sub_ham
    counter += 1
## Eigenvalues
partial_eigvals = ssl.eigsh(partial_hamop.to_matrix(sparse=True),2,which='SA')[0]

print("Selected Pauli strings:", len(pauli_selected))
partial_eigvals, abs(partial_eigvals[0] - true_eigvals[0])

Selected Pauli strings: 39


(array([-231.78734765, -231.63270941]), 0.00043449542994267176)

### Group Pauli Operators and Generate Circuits

In [9]:
import nwq2qis
import importlib
importlib.reload(nwq2qis)
from nwq2qis import *

pauli_dict_partial, coeff_dict_partial = grouped_pauli_in_dict(partial_hamop, use_qwc=True) ## grouping paulis in the Hamiltonian and separate Pauli strings and Coefficients
circ_dict = commuting_circs(list(pauli_dict_partial.keys()), trans_vqe_opt, add_measure = True, 
                            noise_factors=noise_factors, inverse_pauli = False, noise_type='half') ## Form circuits for each commuting group
circ_dict_nomeasure = commuting_circs(list(pauli_dict_partial.keys()), trans_vqe_opt, add_measure = False, 
                            noise_factors=noise_factors, inverse_pauli = False, noise_type='half') ## Form circuits for each commuting group
print("Number of commuting groups:",len(list(pauli_dict_partial.keys())))
# print(f"Total number of cx gates: {trans_vqe_opt.count_ops()['cx']*len(list(pauli_dict_partial.keys()))}")

## Exact Solutions
theo_state = qi.Statevector.from_instruction(vqe_circ.assign_parameters(pre_opt_params)) ## exact state
theo_energy = theo_state.expectation_value(partial_hamop).real
print(theo_energy, partial_eigvals[0], abs(theo_energy - partial_eigvals[0]))

Number of commuting groups: 39
-231.77831320185888 -231.7873476531096 0.009034451250727216


## Qiskit Estimator

In [10]:
from qiskit.primitives import StatevectorEstimator
estimator = StatevectorEstimator()
job = estimator.run([(trans_vqe_opt, partial_hamop)])
job_pre = estimator.run([(trans_vqe_opt, partial_hamop)], precision=1e-8)
float( job.result()[0].data.evs ), float( job_pre.result()[0].data.evs ), true_eigvals[0]-float( job.result()[0].data.evs ), true_eigvals[0]-float( job_pre.result()[0].data.evs ), errors[selected_iter]

(-231.77831320185933,
 -231.77831319814783,
 -0.00946894668021514,
 -0.00946895039172091,
 0.009069664416813339)

In [ ]:
trans_vqe_opt.count_ops()

TZChop_Outputs_QESEM/itr1_qasm


OrderedDict([('u', 22), ('cx', 20), ('barrier', 1)])

## QESEM

In [12]:
exp_obs = partial_hamop
state_prep_circ = trans_vqe_opt

from qiskit.primitives import StatevectorEstimator
estimator = StatevectorEstimator()
rt_exp = estimator.run([(state_prep_circ, exp_obs)]).result()[0].data.evs
float( rt_exp)

-231.77831320185933

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='',
    token=''
)
print(service.backends())
backend = service.backend("ibm_marrakesh")
print(backend.name)

[<IBMBackend('ibm_kingston')>, <IBMBackend('ibm_marrakesh')>, <IBMBackend('ibm_aachen')>, <IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_brussels')>, <IBMBackend('ibm_fez')>, <IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_strasbourg')>, <IBMBackend('ibm_torino')>]
ibm_marrakesh


In [14]:
from qiskit_ibm_catalog import QiskitFunctionsCatalog
catalog = QiskitFunctionsCatalog()
print(catalog.list())
qesem_function = catalog.load("qedma/qesem")

[QiskitFunction(qedma/qesem), QiskitFunction(ibm/circuit-function)]


#### Estimate running time

#### Empirical

In [ ]:
time_estimation_job = qesem_function.run(
    pubs=[(state_prep_circ, [exp_obs])],
    options={
        "default_precision": 0.02, 
        "max_execution_time": 300, 
        "transpilation_level": 1,
        "execution_mode": "batch",  ## "session" / "batch", when batch, QPU is released during classical computation
        "estimate_time_only": "empirical",
    },
    instance='',
    backend_name=backend.name,  # E.g. "ibm_brisbane"
)

print("Time estimation")
print(f">>> Job ID: {time_estimation_job.job_id}")
print(f">>> Job Status: {time_estimation_job.status()}")

Time estimation
>>> Job ID: d860acca-d113-45b9-906b-359a30948dcd
>>> Job Status: QUEUED


In [22]:
time_estimation_job = catalog.get_job_by_id("d860acca-d113-45b9-906b-359a30948dcd")
time_estimation_job.status()

'DONE'

In [23]:
time_estimate_result = time_estimation_job.result()
time_estimate_result

PrimitiveResult([PubResult(data=DataBin(), metadata={'time_estimation_sec': 2100})], metadata={})

## Execution

In [ ]:
exec_job = qesem_function.run(
    pubs=[(state_prep_circ, [exp_obs])],
    options={
        "default_precision": 0.02, 
        "max_execution_time": 3600, 
        "transpilation_level": 1,
        "execution_mode": "batch",  ## "session" / "batch", when batch, QPU is released during classical computation
    },
    instance='',
    backend_name=backend.name,  # E.g. "ibm_brisbane"
)

print("Execution")
print(f">>> Job ID: {exec_job.job_id}")
print(f">>> Job Status: {exec_job.status()}")

Execution
>>> Job ID: 2f4cf7d7-789a-4109-8fa2-507b06602070
>>> Job Status: QUEUED


In [27]:
exec_job.status()

'DONE'

In [28]:
exec_result = exec_job.result() ## catalog.get_job_by_id("2f4cf7d7-789a-4109-8fa2-507b06602070").result()
exec_result

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(1,), dtype=float64>), stds=np.ndarray(<shape=(1,), dtype=float64>), shape=(1,)), metadata={'gate_fidelities': {'CZ': 0.9926880373300168, 'ID1Q': 0.9983025752642868}, 'total_shots': 1927202, 'mitigation_shots': 410002, 'transpiled_circs': [{'circuit': 'OPENQASM 3.0;\ninclude "stdgates.inc";\nbit[129] c0;\nqubit[129] q1;\nrx(1.6118303985790519) q1[92];\nrx(-1.8235805614589236) q1[91];\nrx(-pi/2) q1[111];\nrx(-pi/2) q1[110];\nrx(0) q1[109];\nrx(-pi) q1[108];\nrx(-pi) q1[112];\nrx(pi/4) q1[128];\nrx(-3*pi/4) q1[84];\nrz(pi/2) q1[92];\nrz(-pi/2) q1[91];\nrz(pi/2) q1[111];\nrz(pi/2) q1[110];\nrz(pi/2) q1[109];\nrz(pi/2) q1[108];\nrz(pi/2) q1[112];\nrz(0) q1[128];\nrz(0) q1[84];\nrx(pi/2) q1[92];\nrx(-pi/2) q1[91];\nrx(-pi) q1[111];\nrx(pi/2) q1[110];\nrx(pi/2) q1[109];\nrx(pi/2) q1[108];\nrx(-pi) q1[112];\nrx(-3*pi/4) q1[128];\nrx(pi/4) q1[84];\nbarrier q1[84], q1[91], q1[92], q1[98], q1[108], q1[109], q1[110], q1[111], q1[112], 

In [29]:
exec_result[0].metadata

{'gate_fidelities': {'CZ': 0.9926880373300168, 'ID1Q': 0.9983025752642868},
 'total_shots': 1927202,
 'mitigation_shots': 410002,
 'transpiled_circs': [{'circuit': 'OPENQASM 3.0;\ninclude "stdgates.inc";\nbit[129] c0;\nqubit[129] q1;\nrx(1.6118303985790519) q1[92];\nrx(-1.8235805614589236) q1[91];\nrx(-pi/2) q1[111];\nrx(-pi/2) q1[110];\nrx(0) q1[109];\nrx(-pi) q1[108];\nrx(-pi) q1[112];\nrx(pi/4) q1[128];\nrx(-3*pi/4) q1[84];\nrz(pi/2) q1[92];\nrz(-pi/2) q1[91];\nrz(pi/2) q1[111];\nrz(pi/2) q1[110];\nrz(pi/2) q1[109];\nrz(pi/2) q1[108];\nrz(pi/2) q1[112];\nrz(0) q1[128];\nrz(0) q1[84];\nrx(pi/2) q1[92];\nrx(-pi/2) q1[91];\nrx(-pi) q1[111];\nrx(pi/2) q1[110];\nrx(pi/2) q1[109];\nrx(pi/2) q1[108];\nrx(-pi) q1[112];\nrx(-3*pi/4) q1[128];\nrx(pi/4) q1[84];\nbarrier q1[84], q1[91], q1[92], q1[98], q1[108], q1[109], q1[110], q1[111], q1[112], q1[128];\ncz q1[110], q1[109];\nbarrier q1[110], q1[109];\nbarrier q1[84], q1[91], q1[92], q1[98], q1[108], q1[109], q1[110], q1[111], q1[112], q1[128

In [30]:
exec_result[0].metadata['noisy_results'].__dict__

{'evs': array([-1.28452261]), 'stds': array([0.01488615])}

In [31]:
obs_ps, obs_cs = pauli_coeff_separation(exp_obs)
obs_dict = { pp:cc for pp,cc in zip(obs_ps, obs_cs)}
zero_coeff = obs_dict['I'*12]
full_qesem_exp = zero_coeff + exec_result[0].data['evs'][0]

print(theo_energy, partial_eigvals[0], abs(theo_energy - partial_eigvals[0]))
print(zero_coeff, full_qesem_exp, exec_result[0].data['stds'][0])
print(abs(full_qesem_exp - theo_energy), abs(full_qesem_exp - partial_eigvals[0]), abs(full_qesem_exp - true_eigvals[0]))

-231.77831320185888 -231.7873476531096 0.009034451250727216
-230.18301293165405 -231.7810246544127 0.025905186323406812
0.002711452553825211 0.006322998696902005 0.006757494126844676
